In [233]:
import pymongo
import pandas as pd

In [355]:
client = pymongo.MongoClient("<Your DB Connection>")
db = client.MongoTask
col = db.Students
test_dtl = db.Test_details
bel_avg = db.Below_average
fail = db.failed_students
passed = db.passed_students

#### 1. Find students who scored below average in the exam and pass mark is 40%?

In [289]:
# Finding Average
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$group':
    {
        '_id':'$scores.type',
        'average':{'$avg':'$scores.score'}
    }},
    {'$project':{'_id':1,'average':1}}
])
for i in a:
    print(i)

{'_id': 'exam', 'average': 48.67367075950175}


In [310]:
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$group':
    {
        '_id':'$scores.type',
        'average':{'$avg':'$scores.score'}
    }},
    {'$project':{'_id':0,'average':1}}
])
for i in a:
    average = i['average']
res = col.find({'scores':
                {'$elemMatch':
                  {'$and':
                    [{'type':'exam'},{'score':{'$gte':40,'$lt':average}}]}}},
                   {'_id':0})
for i in res:
    print(i['name'])

Wilburn Spiess
Denisha Cast
Gisela Levin
Tressa Schwing
Rosana Vales
Tamika Schildgen
Gwyneth Garling
Kayce Kenyon
Terica Brugger
Chad Rahe
Jenise Mcguffie
Cassi Heal
Demarcus Audette
Flora Duell
Quincy Danaher
Alix Sherrill
Laureen Salomone
Gwyneth Garling
Aleida Elsass
Alica Pasley
Malisa Jeanes
Harriett Velarde
Mariela Sherer


#### 2. Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [118]:
# exam
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$sort':{'scores.score':-1}},
    {'$project':{'_id':0,'name':1,'scores.score':1}},
    {'$limit':1}
])
for i in a:
    print(i)

{'name': 'Margart Vitello', 'scores': {'score': 99.33685767140612}}


In [119]:
# quiz
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'quiz'}},
    {'$sort':{'scores.score':-1}},
    {'$project':{'_id':0,'name':1,'scores.score':1}},
    {'$limit':1}
])
for i in a:
    print(i)

{'name': 'Cody Strouth', 'scores': {'score': 99.80348240553108}}


In [120]:
# homework
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'homework'}},
    {'$sort':{'scores.score':-1}},
    {'$project':{'_id':0,'name':1,'scores.score':1}},
    {'$limit':1}
])
for i in a:
    print(i)

{'name': 'Whitley Fears', 'scores': {'score': 99.77237745070993}}


#### 3. Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.

In [272]:
# exam
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'exam'}},
    {'$project':{'_id':1,'name':1,'score':'$scores.score'}}
])
for  i in a:
    if i['score'] >= 40:
        ins = col.update_one({'_id':i['_id']},{'$set':{'scores.0.status':'Pass'}})
    else:
        ins = col.update_one({'_id':i['_id']},{'$set':{'scores.0.status':'Fail'}})

In [273]:
# quiz
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'quiz'}},
    {'$project':{'_id':1,'name':1,'score':'$scores.score'}}
])
for  i in a:
    if i['score'] >= 40:
        ins = col.update_one({'_id':i['_id']},{'$set':{'scores.1.status':'Pass'}})
    else:
        ins = col.update_one({'_id':i['_id']},{'$set':{'scores.1.status':'Fail'}})

In [274]:
# homework
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$match':{'scores.type':'homework'}},
    {'$project':{'_id':1,'name':1,'score':'$scores.score'}}
])
for  i in a:
    if i['score'] >= 40:
        ins = col.update_one({'_id':i['_id']},{'$set':{'scores.2.status':'Pass'}})
    else:
        ins = col.update_one({'_id':i['_id']},{'$set':{'scores.2.status':'Fail'}})

#### 4. Find the total and average of the exam, quiz and homework and store them in a separate collection

In [316]:
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$group':
    {
        '_id':'$scores.type',
        'Average':{'$avg':'$scores.score'},
        'Sum' : {'$sum':'$scores.score'}
    }},
    {'$project':{'_id':1,'Average':1,'Sum':1}}
])
for i in a:
    print(i)
    ins = test_dtl.insert_one(i)
    print(ins.acknowledged)

{'_id': 'exam', 'Average': 48.67367075950175, 'Sum': 9734.734151900351}
True
{'_id': 'homework', 'Average': 67.81869620661149, 'Sum': 13563.739241322297}
True
{'_id': 'quiz', 'Average': 48.99672319430254, 'Sum': 9799.344638860508}
True


#### 5. Create a new collection which consists of students who scored below average and above 40% in all the categories.

In [327]:
#exam
a = col.aggregate(
[
    {'$unwind':'$scores'},
    {'$group':
    {
        '_id':'$scores.type',
        'average':{'$avg':'$scores.score'}
    }},
    {'$project':{'_id':1,'average':1}}
])
for i in a:
    if i['_id'] == 'exam':
        exam_avg = i['average']
    elif i['_id'] == 'homework':
        hw_avg = i['average']
    else:
        quiz_avg = i['average']
print('Exam Average = '+str(exam_avg)+' HomeWork Average = '+str(hw_avg)+' Quiz Average = '+str(quiz_avg))

Exam Average = 48.67367075950175 HomeWork Average = 67.81869620661149 Quiz Average = 48.99672319430254


In [345]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'exam','score':{'$gte':40,'$lt':exam_avg}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = bel_avg.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 23


In [346]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'quiz','score':{'$gte':40,'$lt':quiz_avg}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = bel_avg.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 19


In [347]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'homework','score':{'$gte':40,'$lt':hw_avg}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = bel_avg.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 61


#### 6. Create a new collection which consists of students who scored below the fail mark in all the categories.

In [359]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'exam','score':{'$lt':40}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = fail.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 81


In [360]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'quiz','score':{'$lt':40}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = fail.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 86


In [361]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'homework','score':{'$lt':40}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = fail.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 29


#### 7. Create a new collection which consists of students who scored above pass mark in all the categories.

In [362]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'exam','score':{'$gte':40}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = passed.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 119


In [363]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'quiz','score':{'$gte':40}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = passed.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 114


In [364]:
result = col.find(
{
    'scores':{
        '$elemMatch':
        {'$and':[{'type':'homework','score':{'$gte':40}}]},
    }
},{'_id':0})
cnt = 0
for i in result:
    ack = passed.insert_one(i)
    if ack.acknowledged == True:
        cnt +=  1
print('Inserted Documents: '+str(cnt))

Inserted Documents: 171
